In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
//import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
import Resolvers._

import $exclude.$                        , $ivy.$                            // for cleaner logs
//import $profile.`hadoop-2.6`

import $ivy.$                                   // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

import Resolvers._

In [2]:
import $ivy.`org.vegas-viz::vegas:0.3.9`
import vegas._
import vegas.data.External._

import $ivy.$                           

import vegas._

import vegas.data.External._

In [3]:
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "isarn project",
  "https://dl.bintray.com/isarn/maven/",
  MavenPattern,
  true
)
// resolvers += "Will's bintray" at "https://dl.bintray.com/willb/maven/"
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "Will Benton",
  "https://dl.bintray.com/willb/maven/",
  MavenPattern,
  true
)

In [4]:
import $ivy.`org.isarnproject::isarn-sketches:0.0.3.rc1` // "org.isarnproject" %% "isarn-sketches" % "0.0.2"
import org.isarnproject.sketches._

import $ivy.$                                            // "org.isarnproject" %% "isarn-sketches" % "0.0.2"

import org.isarnproject.sketches._

In [5]:
import $ivy.`com.redhat.et::silex:0.1.2` // "com.redhat.et" %% "silex" % "0.1.2"

import $ivy.$                            // "com.redhat.et" %% "silex" % "0.1.2"

In [6]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint

import com.redhat.et.silex.util.vectors.implicits._
import com.redhat.et.silex.cluster.ClusteringTreeModel
import ClusteringTreeModel._

import org.apache.spark.rdd.RDD

import org.apache.spark.mllib.tree.RandomForest

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}

import org.apache.spark.mllib.evaluation.MulticlassMetrics

import org.apache.spark.mllib.regression.LabeledPoint


import com.redhat.et.silex.util.vectors.implicits._

import com.redhat.et.silex.cluster.ClusteringTreeModel

import ClusteringTreeModel._

In [7]:
implicit class TDEnhance(td: TDigest) extends Serializable {
  def toXY: Vector[(Double, Double)] = {
    val q = (0.001 +: (0.05 to 0.95 by 0.05) :+ 0.999).toVector
    q.map(q => (td.cdfInverse(q), q))
  }
}

defined class TDEnhance

In [8]:
val spark = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("spark://frclust:7077")
  .appName("notebook")
  .getOrCreate()

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


spark: SparkSession = org.apache.spark.sql.SparkSession@6355799d

In [9]:
val gaussianData = Vector.fill(1000000) { scala.util.Random.nextGaussian() }
val gaussianRDD = spark.sparkContext.parallelize(gaussianData, 10)

gaussianData: Vector[Double] = Vector(
  -1.1435334582256125,
  -1.2667150729471834,
  -0.27703493488365855,
  0.20932681386120155,
  -0.2084712781527708,
  -0.5427374542900285,
  1.5875049577440887,
  0.7222200654224683,
  1.3792855378501498,
  0.029199759920821855,
  -0.6291287059339786,
...
gaussianRDD: RDD[Double] = ParallelCollectionRDD[0] at parallelize at cmd8.sc:2

In [10]:
val gaussianSketch = gaussianRDD.aggregate(TDigest.empty())(
    (td, x) => td + x,
    (td1, td2) => td1 ++ td2
)

gaussianSketch: TDigest = TDigest(
  0.5,
  0,
  116,
  Map(
    -4.81221346601262 -> 1.0,
    -4.521593281780686 -> 1.0,
    -4.379747991852362 -> 1.0,
    -4.3527813033214136 -> 1.8124925615641077,
    -4.326407228785711 -> 1.1875074384358923,
    -4.260053582930155 -> 3.2499591268180392,
    -4.251002283834409 -> 1.9531049834010576,
...

In [11]:
Vegas("CDF")
    .withXY(gaussianSketch.toXY)
    .encodeX("x", Quant)
    .encodeY("y", Quant)
    .mark(Line)
    .configCell(width=600, height=300)
    .show

<iframe id="frame-vegas-68ab9a24-0e65-4a54-ac14-cfe2bc774b04" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative"
 }
 },
 "description" : "CDF",
 "data" : {
 "values" : [
 {
 "x" : -3.10412820271481,
 "y" : 0.001
 },
 {
 "x" : -1.6384176998552824,
 "y" : 0.05
 },
 {
 "x" : -1.2814862655348531,
 "y" : 0.1
 },
 {
 "x" : -1.0258460477035294,
 "y" : 0.15000000000000002
 },
 {
 "x" : -0.8408192282565279,
 "y" : 0.2
 },
 {
 "x" : -0.6747677103728483,
 "y" : 0.25
 },
 {
 "x" : -0.5257179355839212,
 "y" : 0.3
 },
 {
 "x" : -0.3861805783094846,
 "y" : 0.35
 },
 {
 "x" : -0.25181896390015013,
 "y" : 0.39999999999999997
 },
 {
 "x" : -0.12595385021594122,
 "y" : 0.44999999999999996
 },
 {
 "x" : -3.693776689685513E-4,
 "y" : 0.49999999999999994
 },
 {
 "x" : 0.12500721905196527,
 "y" : 0.5499999999999999
 },
 {
 "x" : 0.2528028937725217,
 "y" : 0.6
 },
 {
 "x" : 0.3848135587475399,
 "y" : 0.65
 },
 {
 "x" : 0.5258053156238164,
 "y" : 0.7000000000000001
 },
 {
 "x" : 0.6728690513169981,
 "y" : 0.7500000000000001
 },
 {
 "x" : 0.8411765055484939,
 "y" : 0.8000000000000002
 },
 {
 "x" : 1.0349769311195014,
 "y" : 0.8500000000000002
 },
 {
 "x" : 1.2826563918967144,
 "y" : 0.9000000000000002
 },
 {
 "x" : 3.0835057673501303,
 "y" : 0.999
 }
 ]
 },
 "config" : {
 "cell" : {
 "width" : 600.0,
 "height" : 300.0
 }
 }
}
 }
 vg.embed("#vegas-68ab9a24-0e65-4a54-ac14-cfe2bc774b04", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-68ab9a24-0e65-4a54-ac14-cfe2bc774b04'></div>
 </body>
</html>">

In [12]:
def loadCSV(fname: String) = spark.read
  .format("com.databricks.spark.csv")
  .option("header", "true")
  .option("mode", "DROPMALFORMED")
  .load(fname)

defined function loadCSV

In [13]:
val trainCSV = loadCSV("/data/tox21/tox21_dense_train.csv")
val testCSV = loadCSV("/data/tox21/tox21_dense_test.csv")
val trainLabelsCSV = loadCSV("/data/tox21/tox21_labels_train.csv")
val testLabelsCSV = loadCSV("/data/tox21/tox21_labels_test.csv")

trainCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]
testCSV: DataFrame = [_c0: string, AW: string ... 800 more fields]
trainLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]
testLabelsCSV: DataFrame = [_c0: string, NR.AhR: string ... 11 more fields]

In [14]:
def tdSketchFV(fv: RDD[Vector[Double]]): Vector[TDigest] = {
    val tds = fv.aggregate(Array.empty[TDigest])(
        (tdv, xv) => {
            if (tdv.isEmpty) {
                Array.tabulate(xv.length) { j => TDigest.empty(maxDiscrete=50) + xv(j) }
            } else {
                for { j <- 0 until xv.length } { tdv(j) += xv(j) }
                tdv
            }
        },
        (tdv1, tdv2) => {
            if (tdv1.isEmpty) tdv2 else {
                for { j <- 0 until tdv1.length } { tdv1(j) ++= tdv2(j) }
                tdv1
            }
        }
    )
    tds.toVector
}

defined function tdSketchFV

In [15]:
def synthesize(tdVec: Vector[TDigest], n: Int, partitions: Int = 2) = {
    implicit class AddSampling(td: TDigest) {
        import org.isarnproject.collections.mixmaps.nearest.Cover
        def cdfDiscreteInverse(q: Double): Double = {
            td.clusters.mCover(q * td.clusters.sum).map(n => (n.data.key, n.data.value)) match {
                case Cover(_, Some((x, _))) => x
                case Cover(Some((x, _)), None) => x
            }
        }

        def sample: Double = {
            val clust = td.clusters
            td.nclusters match {
                case 0 => 0.0
                case 1 => clust(clust.keyMin.get)
                case n if (n <= td.maxDiscrete) => cdfDiscreteInverse(scala.util.Random.nextDouble)
                case _ => td.cdfInverse(scala.util.Random.nextDouble)
            }
        }
    }
    val tdVecBC = spark.sparkContext.broadcast(tdVec)
    spark.sparkContext.parallelize(1 to n, partitions).map { r => tdVecBC.value.map(_.sample) }
}

defined function synthesize

In [16]:
val trainFV = trainCSV.rdd.map(_.toSeq.toVector.drop(1).map(_.asInstanceOf[String].toDouble))
val fvSketches = tdSketchFV(trainFV)

trainFV: RDD[Vector[Double]] = MapPartitionsRDD[33] at map at cmd15.sc:1
fvSketches: Vector[TDigest] = Vector(
  TDigest(
    0.5,
    50,
    75,
    Map(
      1.0 -> 9.0,
      1.333 -> 24.0,
      1.5 -> 45.0,
      1.600489455071117 -> 33.242848970251714,
      1.675738883726358 -> 7.919011523893346,
      1.6979657626987033 -> 63.83813950585494,
...

In [17]:
val synthFV = synthesize(fvSketches, 48000, 4)

synthFV: RDD[Vector[Double]] = MapPartitionsRDD[35] at map at cmd14.sc:22

In [18]:
val trainLab = trainFV.map(_.toLabeledPoint(1.0))
val synthLab = synthFV.map(_.toLabeledPoint(0.0))
val trainFR = (trainLab ++ synthLab).cache()

trainLab: RDD[LabeledPoint] = MapPartitionsRDD[36] at map at cmd17.sc:1
synthLab: RDD[LabeledPoint] = MapPartitionsRDD[37] at map at cmd17.sc:2
trainFR: RDD[LabeledPoint] = UnionRDD[38] at $plus$plus at cmd17.sc:3

In [19]:
def frClassifier = {
val rfFR = RandomForest.trainClassifier(
    trainFR,              // training data
    2,                    // number of classes
    Map.empty[Int, Int],  // category info
    100,                  // forest size
    "auto",               // 
    "gini",               // split quality measure
    10,                   // max depth
    20,                   // max bins
    235711)               // random seed
val predictionAndLabels = trainLab.map { case LabeledPoint(label, features) =>
  val prediction = rfFR.predict(features)
  (prediction, label)
}
val metrics = new MulticlassMetrics(predictionAndLabels)
(rfFR, metrics)
}

defined function frClassifier

In [20]:
val (rfFR, metrics) = frClassifier
metrics.accuracy

rfFR: org.apache.spark.mllib.tree.model.RandomForestModel = TreeEnsembleModel classifier with 100 trees

metrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@3863a938
res19_1: Double = 1.0

In [21]:
val frRules = rfFR.trees.map(_.rules(trainCSV.columns.toVector.drop(1), Map.empty[Int, Int]))

frRules: Array[Map[Double, Seq[Seq[Predicate]]]] = Array(
  Map(
    1.0 -> ArrayBuffer(
      List((MRVSA2 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 > 0.0), (PEOEVSA4 <= 0.0)),
      List((MRVSA2 > 0.0), (RDFM30 > 0.0), (PEOEVSA4 > 0.0), (Chiv4c <= 0.0)),
      List(
        (MRVSA2 > 0.0),
        (RDFM30 > 0.0),
        (PEOEVSA4 > 0.0),
        (Chiv4c > 0.0),
...

In [22]:
val featPerTree = for {
    rmap <- frRules
    feats = {
        val fraw = for {
            vrules <- rmap.values
            plist <- vrules
            pred <- plist
        } yield { pred.feature }
        fraw.toSet.toSeq
    }
    f <- feats
} yield { f }
val featCounts = featPerTree.foldLeft(Map.empty[String, Int])((m, f) => m + (f -> (1 + m.getOrElse(f, 0))))
val featHist = featCounts.toSeq.sortBy { case (_, n) => -n}.toVector
println(featHist.mkString("\n"))

(MRVSA7,22)
(VSAEstate2,20)
(RNCS,20)
(VSAEstate0,18)
(VSAEstate3,17)
(slogPVSA8,17)
(RDFM29,13)
(PEOEVSA4,13)
(slogPVSA3,12)
(slogPVSA6,12)
(RDFV30,11)
(RDFM30,11)
(RDFU27,11)
(RDFE30,10)
(Chi4c,9)
(PEOEVSA3,9)
(MRVSA8,8)
(Chiv4c,8)
(RDFV24,8)
(MRVSA2,8)
(RDFU30,8)
(PEOEVSA12,7)
(PEOEVSA11,7)
(slogPVSA9,7)
(RDFV29,7)
(RDFP30,7)
(RDFM24,7)
(PEOEVSA10,7)
(Chi5ch,6)
(RDFP27,6)
(slogPVSA0,6)
(MRVSA3,6)
(RDFM27,6)
(RDFE28,6)
(RDFE29,6)
(EstateVSA5,5)
(RDFV23,5)
(RDFM25,5)
(RDFV27,5)
(RDFP29,5)
(slogPVSA7,5)
(RDFU29,5)
(RDFU25,5)
(PEOEVSA2,4)
(slogPVSA11,4)
(RDFE26,4)
(EstateVSA2,4)
(EstateVSA6,4)
(Chiv5ch,4)
(RDFV25,4)
(RDFE27,4)
(RDFV28,4)
(RDFU26,4)
(MRVSA5,3)
(RDFP24,3)
(RDFM22,3)
(EstateVSA1,3)
(RDFP21,3)
(RDFP26,3)
(Tv,3)
(PEOEVSA8,3)
(RDFP28,3)
(PEOEVSA13,3)
(slogPVSA2,3)
(L1p,2)
(EstateVSA8,2)
(RDFU28,2)
(GATSm8,2)
(Ve,2)
(RDFE20,2)
(J,2)
(rygr,2)
(RDFM28,2)
(RDFE22,2)
(PEOEVSA1,2)
(ncarb,2)
(Tu,2)
(RDFM23,2)
(EstateVSA7,2)
(RDFE25,2)
(EstateVSA4,2)
(RDFM20,2)
(RDFE21,2)
(EstateVSA9

featPerTree: Array[String] = Array(
  "PEOEVSA12",
  "RDFM29",
  "RDFP27",
  "RDFM30",
  "VSAEstate3",
  "Chiv4c",
  "MRVSA2",
  "RDFV20",
  "MoRSEP22",
  "RDFE27",
  "PEOEVSA4",
...
featCounts: Map[String, Int] = Map(
  "MRVSA8" -> 8,
  "PEOEVSA12" -> 7,
  "nta" -> 1,
  "VSAEstate2" -> 20,
  "MRVSA5" -> 3,
  "Chi5ch" -> 6,
  "RDFM2" -> 1,
  "RDFM29" -> 13,
  "RDFU17" -> 1,
  "MoRSEN9" -> 1,
  "RDFP27" -> 6,
...
featHist: Vector[(String, Int)] = Vector(
  ("MRVSA7", 22),
  ("VSAEstate2", 20),
  ("RNCS", 20),
  ("VSAEstate0", 18),
  ("VSAEstate3", 17),
  ("slogPVSA8", 17),
  ("RDFM29", 13),
  ("PEOEVSA4", 13),
  ("slogPVSA3", 12),
  ("slogPVSA6", 12),
  ("RDFV30", 11),
...

In [23]:
val featSelect = featHist.filter { case (_, n) => n > 1 }.map { case (f, _) => f }
featSelect.length
featHist.length

featSelect: Vector[String] = Vector(
  "MRVSA7",
  "VSAEstate2",
  "RNCS",
  "VSAEstate0",
  "VSAEstate3",
  "slogPVSA8",
  "RDFM29",
  "PEOEVSA4",
  "slogPVSA3",
  "slogPVSA6",
  "RDFV30",
...
res22_1: Int = 84
res22_2: Int = 146

In [24]:
val trainCSVFR = trainCSV.select("_c0", featSelect :_*)
val testCSVFR = testCSV.select("_c0", featSelect :_*)

trainCSVFR: DataFrame = [_c0: string, MRVSA7: string ... 83 more fields]
testCSVFR: DataFrame = [_c0: string, MRVSA7: string ... 83 more fields]

In [25]:
println(trainLabelsCSV.columns.mkString(", "))

_c0, NR.AhR, NR.AR, NR.AR.LBD, NR.Aromatase, NR.ER, NR.ER.LBD, NR.PPAR.gamma, SR.ARE, SR.ATAD5, SR.HSE, SR.MMP, SR.p53


In [26]:
def csvToPairs(csv: DataFrame) =
    csv.rdd.map(_.toSeq)
    .map(v => (v.head.asInstanceOf[String], v.tail.toVector.map(_.asInstanceOf[String].toDouble)))
def lpFR(tlab: String, labCSV: DataFrame, fvCSV: DataFrame) = {
    val labCol = labCSV.select("_c0", tlab)
    val labPairs = labCol.filter(labCol(tlab) =!= "NA")
        .rdd.map(_.toSeq).map { s => (s(0).asInstanceOf[String], s(1).asInstanceOf[String].toDouble) }
    val fvPairs = csvToPairs(fvCSV)
    val lp = labPairs.join(fvPairs).map { case (_, (lab, feats)) => LabeledPoint(lab, feats.toSpark) }
    lp
}
def trainAndTest(tlab: String) =
  (lpFR(tlab, trainLabelsCSV, trainCSVFR), lpFR(tlab, testLabelsCSV, testCSVFR))

defined function csvToPairs
defined function lpFR
defined function trainAndTest

In [27]:
def trainLRModel(train: RDD[LabeledPoint], test: RDD[LabeledPoint]) = {
    val lrModel = new LogisticRegressionWithLBFGS().run(train)
    val predictionAndLabels = test.map { case LabeledPoint(label, features) =>
        val prediction = lrModel.predict(features)
        (prediction, label)
    }
    val metrics = new MulticlassMetrics(predictionAndLabels)
    (lrModel, predictionAndLabels, metrics)
}

defined function trainLRModel

In [28]:
val allvsFR = trainLabelsCSV.columns.tail.map { acol => (acol, s"`$acol`")}
    .flatMap { case (acolRaw, acol) =>
        val (trainFR, testFR) = (lpFR(acol, trainLabelsCSV, trainCSVFR), lpFR(acol, testLabelsCSV, testCSVFR))
        val (trainAll, testAll) = (lpFR(acol, trainLabelsCSV, trainCSV), lpFR(acol, testLabelsCSV, testCSV))
        val (modelFR, predLabFR, metricsFR) = trainLRModel(trainFR, testFR)
        val (modelAll, predLabAll, metricsAll) = trainLRModel(trainAll, testAll)
        Seq(
            Map(
                "assay" -> acolRaw,
                "feats" -> s"${modelAll.numFeatures}",
                "accuracy" -> metricsAll.accuracy
            ),
            Map(
                "assay" -> acolRaw,
                "feats" -> s"${modelFR.numFeatures}",
                "accuracy" -> metricsFR.accuracy
            )
        )
    }

allvsFR: Array[Map[String, Any]] = Array(
  Map("assay" -> NR.AhR, "feats" -> 801, "accuracy" -> 0.8819672131147541),
  Map("assay" -> NR.AhR, "feats" -> 84, "accuracy" -> 0.8786885245901639),
  Map("assay" -> NR.AR, "feats" -> 801, "accuracy" -> 0.9744027303754266),
  Map("assay" -> NR.AR, "feats" -> 84, "accuracy" -> 0.9761092150170648),
  Map("assay" -> NR.AR.LBD, "feats" -> 801, "accuracy" -> 0.9725085910652921),
  Map("assay" -> NR.AR.LBD, "feats" -> 84, "accuracy" -> 0.9845360824742269),
  Map("assay" -> NR.Aromatase, "feats" -> 801, "accuracy" -> 0.9185606060606061),
  Map("assay" -> NR.Aromatase, "feats" -> 84, "accuracy" -> 0.9204545454545454),
  Map("assay" -> NR.ER, "feats" -> 801, "accuracy" -> 0.9011627906976745),
  Map("assay" -> NR.ER, "feats" -> 84, "accuracy" -> 0.9031007751937985),
...

In [29]:
Vegas().
  withData(allvsFR).
  mark(Bar).
  encodeColumn("assay", Nominal, scale=Scale(padding=20.0),
               axis=Axis(title="tox assay", orient=Orient.Bottom, axisWidth=1.0, labelAngle=45,labelAlign="bottom",offset= -5)).
  encodeY("accuracy", Quantitative, axis=Axis(title="model accuracy", grid=false)).
  encodeX("feats", Nominal, scale=Scale(bandSize = 6.0), hideAxis=true).
  encodeColor("feats", Nominal, scale=Scale(rangeNominals=List("#EA98D2", "#659CCA"))).
  configFacet(cell=CellConfig(strokeWidth = 0,width=600)).
  configCell(width=600, height=300).
  show

<iframe id="frame-vegas-043b89d0-c738-40fb-a9dd-59f179b1c3e8" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "column" : {
 "axis" : {
 "labelAngle" : 45.0,
 "orient" : "bottom",
 "title" : "tox assay",
 "axisWidth" : 1.0,
 "offset" : -5.0,
 "labelAlign" : "bottom"
 },
 "scale" : {
 "padding" : 20.0
 },
 "field" : "assay",
 "type" : "nominal"
 },
 "x" : {
 "axis" : false,
 "scale" : {
 "bandSize" : 6.0
 },
 "field" : "feats",
 "type" : "nominal"
 },
 "y" : {
 "axis" : {
 "title" : "model accuracy",
 "grid" : false
 },
 "field" : "accuracy",
 "type" : "quantitative"
 },
 "color" : {
 "scale" : {
 "range" : [
 "#EA98D2",
 "#659CCA"
 ]
 },
 "field" : "feats",
 "type" : "nominal"
 }
 },
 "data" : {
 "values" : [
 {
 "assay" : "NR.AhR",
 "feats" : "801",
 "accuracy" : 0.8819672131147541
 },
 {
 "assay" : "NR.AhR",
 "feats" : "84",
 "accuracy" : 0.8786885245901639
 },
 {
 "assay" : "NR.AR",
 "feats" : "801",
 "accuracy" : 0.9744027303754266
 },
 {
 "assay" : "NR.AR",
 "feats" : "84",
 "accuracy" : 0.9761092150170648
 },
 {
 "assay" : "NR.AR.LBD",
 "feats" : "801",
 "accuracy" : 0.9725085910652921
 },
 {
 "assay" : "NR.AR.LBD",
 "feats" : "84",
 "accuracy" : 0.9845360824742269
 },
 {
 "assay" : "NR.Aromatase",
 "feats" : "801",
 "accuracy" : 0.9185606060606061
 },
 {
 "assay" : "NR.Aromatase",
 "feats" : "84",
 "accuracy" : 0.9204545454545454
 },
 {
 "assay" : "NR.ER",
 "feats" : "801",
 "accuracy" : 0.9011627906976745
 },
 {
 "assay" : "NR.ER",
 "feats" : "84",
 "accuracy" : 0.9031007751937985
 },
 {
 "assay" : "NR.ER.LBD",
 "feats" : "801",
 "accuracy" : 0.96
 },
 {
 "assay" : "NR.ER.LBD",
 "feats" : "84",
 "accuracy" : 0.96
 },
 {
 "assay" : "NR.PPAR.gamma",
 "feats" : "801",
 "accuracy" : 0.9289256198347108
 },
 {
 "assay" : "NR.PPAR.gamma",
 "feats" : "84",
 "accuracy" : 0.9388429752066115
 },
 {
 "assay" : "SR.ARE",
 "feats" : "801",
 "accuracy" : 0.8162162162162162
 },
 {
 "assay" : "SR.ARE",
 "feats" : "84",
 "accuracy" : 0.827027027027027
 },
 {
 "assay" : "SR.ATAD5",
 "feats" : "801",
 "accuracy" : 0.9389067524115756
 },
 {
 "assay" : "SR.ATAD5",
 "feats" : "84",
 "accuracy" : 0.9356913183279743
 },
 {
 "assay" : "SR.HSE",
 "feats" : "801",
 "accuracy" : 0.9573770491803278
 },
 {
 "assay" : "SR.HSE",
 "feats" : "84",
 "accuracy" : 0.9590163934426229
 },
 {
 "assay" : "SR.MMP",
 "feats" : "801",
 "accuracy" : 0.8931860036832413
 },
 {
 "assay" : "SR.MMP",
 "feats" : "84",
 "accuracy" : 0.8692449355432781
 },
 {
 "assay" : "SR.p53",
 "feats" : "801",
 "accuracy" : 0.922077922077922
 },
 {
 "assay" : "SR.p53",
 "feats" : "84",
 "accuracy" : 0.9253246753246753
 }
 ]
 },
 "config" : {
 "cell" : {
 "width" : 600.0,
 "height" : 300.0
 },
 "facet" : {
 "cell" : {
 "width" : 600.0,
 "strokeWidth" : 0.0
 }
 }
 }
}
 }
 vg.embed("#vegas-043b89d0-c738-40fb-a9dd-59f179b1c3e8", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-043b89d0-c738-40fb-a9dd-59f179b1c3e8'></div>
 </body>
</html>">